<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=57cc0b63a931837d11bfbd834984d9d5b07bc0f6f7025c60118e52a3401ec46b
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf


    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-14 13:17:57
-------------------
qualified stocks: 87
with latest results: 26
still star stocks: 16
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  3.00 K
Current:  1.45 C
-------------------
Today PnL: -67.43 K (-0.46%)
Current PnL: -20.47 L (-13.41%)
CY Booked + Current PnL: -6.95 L (-4.56%)
-------------------
Total profit:  1.79 L
Total loss:  -22.26 L
-------------------
Total Booked + Current PnL: 20.03 L (15.94%)
Total Booked PnL: 40.50 L (32.24%)
Curr Year Booked PnL: 13.52 L (9.33%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.36 C
Est FTT PnL: 91.36 L (63.03%)
Deployed:  1.26 C
Current:  1.45 C
CAGR/XIRR %: 8.31%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.00,101.71,51.0,M-SC,2.69,86866.0,-13911.0,14003.0,-0.14,-13.80,16.12,0.09,253.0,-0.99,0.60,14.12,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.00,144.42,84.0,M-SC,3.12,160158.0,-14401.0,14430.0,4.79,-8.25,9.01,0.02,234.0,-1.00,1.11,39.31,OX40N,NTT,PAINTS
2,ABBOTINDIA,35195.00,-15.95,43.0,X-MC,3.39,87855.0,-2823.0,17729.0,-0.96,-3.11,20.18,16.44,63.0,-0.16,0.61,16.43,X40,ATH,PHARMA
43,ITC,452.00,-40.74,45.0,X-LC,0.88,198632.0,-1506.0,22406.0,-0.20,-0.75,11.28,10.44,5.0,-0.07,1.37,4.77,X40,NTT,FMCG
35,ICICIGI,2252.93,-16.25,54.0,X-MC,7.50,186502.0,12182.0,23014.0,-1.14,6.99,12.34,20.19,68.0,0.53,1.29,23.16,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
69,STARHEALTH,761.00,37.96,72.0,H-SC,15.25,284349.0,3226.0,129635.0,5.04,1.15,45.59,47.26,171.0,0.02,1.96,52.35,XY24,NTT,INSURANCE
39,INDIGOPNTS,1408.00,144.42,84.0,M-SC,3.12,160158.0,-14401.0,14430.0,4.79,-8.25,9.01,0.02,234.0,-1.00,1.11,39.31,OX40N,NTT,PAINTS
6,ASIANPAINT,3465.66,0.35,86.0,X-LC,8.02,255068.0,3300.0,49917.0,4.65,1.31,19.57,21.13,31.0,0.07,1.76,37.20,X40,ATH,PAINTS
13,BERGEPAINT,680.00,-4.44,70.0,X-MC,1.62,233705.0,6366.0,41693.0,3.03,2.80,17.84,21.14,74.0,0.15,1.61,31.73,XY24,NTT,PAINTS
56,RAJESHEXPO,518.00,1703.03,57.0,L-SC,3.47,49197.0,-87980.0,88072.0,2.86,-64.14,179.02,0.07,268.0,-1.00,0.34,22.36,OX40N,NTT,JEWELLERY


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,REPCOHOME,880.00,-55.29,52.0,H-SC,9.47,257737.0,-28768.0,286114.0,-5.12,-10.04,111.01,89.82,160.0,-0.10,1.78,36.36,XY24,NTT,FINANCE
41,INFY,2275.00,-20.67,52.0,X-LC,4.39,318265.0,5377.0,166325.0,-3.71,1.72,52.26,54.87,2.0,0.03,2.20,10.14,X40,BTT,IT
68,SONACOMS,806.63,-34.90,56.0,M-SC,6.32,84166.0,-16993.0,56997.0,-3.38,-16.80,67.72,39.54,220.0,-0.30,0.58,18.68,AR,ATH,AUTO
55,QUESS,986.00,-53.17,28.0,X-SC,37.32,46465.0,-18541.0,169467.0,-3.30,-28.52,364.72,232.18,83.0,-0.11,0.32,0.00,XY24,NTT,MISC
48,KPIGREEN,731.05,-12.49,38.0,H-SC,10.16,118692.0,-6605.0,65530.0,-3.07,-5.27,55.21,47.03,154.0,-0.10,0.82,48.61,MH,ATH,POWER


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WHIRLPOOL,2270.00,-41.57,39.0,M-SC,9.16,114448.0,-350.0,94397.0,1.39,-0.30,82.48,81.92,236.0,-0.00,0.79,35.49,XR,NTT,DURABLES
25,FINCABLES,1641.55,-19.34,44.0,M-SC,10.45,146253.0,-1391.0,160717.0,-0.80,-0.94,109.89,107.91,216.0,-0.01,1.01,2.03,OX40N,ATH,CABLES
86,ZYDUSLIFE,1286.17,-14.53,37.0,H-MC,5.88,208591.0,-983.0,75656.0,-0.79,-0.47,36.27,35.63,119.0,-0.01,1.44,16.12,AR,ATH,PHARMA
85,WIPRO,420.00,-14.25,51.0,M-LC,5.50,150660.0,-285.0,109741.0,-0.91,-0.19,72.84,72.51,99.0,-0.00,1.04,5.82,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-19.34,44.0,M-SC,10.45,146253.0,-1391.0,160717.0,-0.80,-0.94,109.89,107.91,216.0,-0.01,1.01,2.03,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,144.42,84.0,M-SC,3.12,160158.0,-14401.0,14430.0,4.79,-8.25,9.01,0.02,234.0,-1.00,1.11,39.31,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,101.71,51.0,M-SC,2.69,86866.0,-13911.0,14003.0,-0.14,-13.80,16.12,0.09,253.0,-0.99,0.60,14.12,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-65.79,53.0,H-SC,2.41,224757.0,-44910.0,81250.0,1.71,-16.65,36.15,13.47,158.0,-0.55,1.55,14.28,XY24,NTT,PAINTS
67,SIS,528.00,2024.35,48.0,H-SC,2.42,85731.0,-25301.0,48910.0,-0.21,-22.79,57.05,21.26,166.0,-0.52,0.59,15.83,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
69,STARHEALTH,761.00,37.96,72.0,H-SC,15.25,284349.0,3226.0,129635.0,5.04,1.15,45.59,47.26,171.0,0.02,1.96,52.35,XY24,NTT,INSURANCE
62,SAMMAANCAP,326.00,17.65,53.0,M-SC,36.02,158652.0,9432.0,134743.0,-0.39,6.32,84.93,96.62,245.0,0.07,1.09,71.70,XY25,NTT,FINANCE
38,INDIAMART,4810.62,-47.96,51.0,H-SC,4.74,129850.0,6514.0,125110.0,-2.18,5.28,96.35,106.72,139.0,0.05,0.90,29.89,AR,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,-47.96,51.0,H-SC,4.74,129850.0,6514.0,125110.0,-2.18,5.28,96.35,106.72,139.0,0.05,0.90,29.89,AR,ATH,MISC
84,WHIRLPOOL,2270.00,-41.57,39.0,M-SC,9.16,114448.0,-350.0,94397.0,1.39,-0.30,82.48,81.92,236.0,-0.00,0.79,35.49,XR,NTT,DURABLES
85,WIPRO,420.00,-14.25,51.0,M-LC,5.50,150660.0,-285.0,109741.0,-0.91,-0.19,72.84,72.51,99.0,-0.00,1.04,5.82,XR,NTT,IT
86,ZYDUSLIFE,1286.17,-14.53,37.0,H-MC,5.88,208591.0,-983.0,75656.0,-0.79,-0.47,36.27,35.63,119.0,-0.01,1.44,16.12,AR,ATH,PHARMA
25,FINCABLES,1641.55,-19.34,44.0,M-SC,10.45,146253.0,-1391.0,160717.0,-0.80,-0.94,109.89,107.91,216.0,-0.01,1.01,2.03,OX40N,ATH,CABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BATAINDIA,2096.00,11.91,27.0,X-SC,9.02,85366.0,-43304.0,88601.0,-2.74,-33.66,103.79,35.20,92.0,-0.49,0.59,0.00,X40,NTT,FOOTWEAR
55,QUESS,986.00,-53.17,28.0,X-SC,37.32,46465.0,-18541.0,169467.0,-3.30,-28.52,364.72,232.18,83.0,-0.11,0.32,0.00,XY24,NTT,MISC
26,GILLETTE,11206.78,-17.34,29.0,X-SC,5.71,255285.0,1389.0,80925.0,2.72,0.55,31.70,32.42,85.0,0.02,1.76,15.73,X40,ATH,FMCG
27,GLAXO,3466.20,-19.65,29.0,X-MC,5.35,199592.0,308.0,77701.0,-0.47,0.15,38.93,39.15,60.0,0.00,1.38,29.20,X40,ATH,PHARMA
46,JSWINFRA,342.00,-19.06,31.0,X-MC,5.37,189338.0,-11009.0,41522.0,-0.30,-5.49,21.93,15.23,66.0,-0.27,1.31,24.98,X40N,NTT,REALTY


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TMPV,1065.00,-16.97,38.0,X-LC,0.08,256727.0,-19683.0,440852.0,-1.51,-7.12,171.72,152.37,3.0,-0.04,1.77,0.28,XY24,NTT,AUTO
36,ICICIPRULI,790.00,-20.34,66.0,X-MC,0.78,184165.0,7427.0,47312.0,-0.70,4.20,25.69,30.97,75.0,0.16,1.27,17.29,X40,ATH,INSURANCE
43,ITC,452.00,-40.74,45.0,X-LC,0.88,198632.0,-1506.0,22406.0,-0.20,-0.75,11.28,10.44,5.0,-0.07,1.37,4.77,X40,NTT,FMCG
34,HONAUT,58357.33,-25.10,35.0,X-SC,0.97,106410.0,-21528.0,68666.0,-0.07,-16.83,64.53,36.84,90.0,-0.31,0.73,8.94,X40N,ATH,ELECTRICAL
16,CAMS,4762.00,-1.69,56.0,X-SC,1.42,259670.0,7814.0,54609.0,-0.75,3.10,21.03,24.79,86.0,0.14,1.79,26.96,X40N,NTT,MISC


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BATAINDIA,2096.00,11.91,27.0,X-SC,9.02,85366.0,-43304.0,88601.0,-2.74,-33.66,103.79,35.20,92.0,-0.49,0.59,0.00,X40,NTT,FOOTWEAR
55,QUESS,986.00,-53.17,28.0,X-SC,37.32,46465.0,-18541.0,169467.0,-3.30,-28.52,364.72,232.18,83.0,-0.11,0.32,0.00,XY24,NTT,MISC
76,TMPV,1065.00,-16.97,38.0,X-LC,0.08,256727.0,-19683.0,440852.0,-1.51,-7.12,171.72,152.37,3.0,-0.04,1.77,0.28,XY24,NTT,AUTO
53,PAGEIND,51769.93,-28.46,37.0,X-MC,7.63,159080.0,-4660.0,47994.0,-2.33,-2.85,30.17,26.47,55.0,-0.10,1.10,0.79,X40,ATH,APPARELS
10,BAJAJHFL,181.50,-18.37,36.0,X-MC,6.44,176349.0,-24157.0,121310.0,-0.46,-12.05,68.79,48.45,64.0,-0.20,1.22,1.69,X40N,ATH,FINANCE


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,986.00,-53.17,28.0,X-SC,37.32,46465.0,-18541.0,169467.0,-3.30,-28.52,364.72,232.18,83.0,-0.11,0.32,0.00,XY24,NTT,MISC
59,RELAXO,1176.00,-41.83,37.0,X-SC,3.63,79336.0,-65824.0,141750.0,-2.90,-45.35,178.67,52.31,91.0,-0.46,0.55,6.20,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,11.91,27.0,X-SC,9.02,85366.0,-43304.0,88601.0,-2.74,-33.66,103.79,35.20,92.0,-0.49,0.59,0.00,X40,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-15.95,43.0,X-MC,3.39,87855.0,-2823.0,17729.0,-0.96,-3.11,20.18,16.44,63.0,-0.16,0.61,16.43,X40,ATH,PHARMA
34,HONAUT,58357.33,-25.10,35.0,X-SC,0.97,106410.0,-21528.0,68666.0,-0.07,-16.83,64.53,36.84,90.0,-0.31,0.73,8.94,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4389.97,-26.86,56.0,X-LC,8.77,289764.0,-56192.0,122889.0,-1.57,-16.24,42.41,19.28,1.0,-0.46,2.00,7.12,X40,ATH,IT
41,INFY,2275.00,-20.67,52.0,X-LC,4.39,318265.0,5377.0,166325.0,-3.71,1.72,52.26,54.87,2.0,0.03,2.20,10.14,X40,BTT,IT
76,TMPV,1065.00,-16.97,38.0,X-LC,0.08,256727.0,-19683.0,440852.0,-1.51,-7.12,171.72,152.37,3.0,-0.04,1.77,0.28,XY24,NTT,AUTO
82,VBL,671.64,-18.58,46.0,X-LC,5.86,292108.0,-23734.0,136385.0,-0.32,-7.51,46.69,35.67,4.0,-0.17,2.02,5.30,X40N,ATH,FMCG
43,ITC,452.00,-40.74,45.0,X-LC,0.88,198632.0,-1506.0,22406.0,-0.20,-0.75,11.28,10.44,5.0,-0.07,1.37,4.77,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COFFEEDAY,80.00,-47.69,42.0,L-SC,25.81,76531.0,-37018.0,77067.0,-1.46,-32.60,100.70,35.27,270.0,-0.48,0.53,87.31,XR,NTT,HOTELS
7,ASIANTILES,137.00,7555.56,64.0,L-SC,15.83,84020.0,-9790.0,86406.0,-1.62,-10.44,102.84,81.67,271.0,-0.11,0.58,62.71,XR,NTT,CERAMICS
51,MASFIN,398.61,-18.25,52.0,H-SC,7.53,92655.0,-5325.0,26926.0,0.28,-5.43,29.06,22.05,164.0,-0.20,0.64,34.68,XR,ATH,FINANCE
80,VAIBHAVGBL,521.00,71.90,60.0,H-SC,7.52,149009.0,-33766.0,144837.0,-0.26,-18.47,97.20,60.77,159.0,-0.23,1.03,34.19,XR,NTT,JEWELLERY
65,SHALBY,327.00,1064.27,38.0,M-SC,16.43,158274.0,-22869.0,68343.0,-0.18,-12.62,43.18,25.10,250.0,-0.33,1.09,26.84,XY24,NTT,HEALTHCARE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANPAINT,3465.66,0.35,86.0,X-LC,8.02,255068.0,3300.0,49917.0,4.65,1.31,19.57,21.13,31.0,0.07,1.76,37.20,X40,ATH,PAINTS
39,INDIGOPNTS,1408.00,144.42,84.0,M-SC,3.12,160158.0,-14401.0,14430.0,4.79,-8.25,9.01,0.02,234.0,-1.00,1.11,39.31,OX40N,NTT,PAINTS
80,VAIBHAVGBL,521.00,71.90,60.0,H-SC,7.52,149009.0,-33766.0,144837.0,-0.26,-18.47,97.20,60.77,159.0,-0.23,1.03,34.19,XR,NTT,JEWELLERY
7,ASIANTILES,137.00,7555.56,64.0,L-SC,15.83,84020.0,-9790.0,86406.0,-1.62,-10.44,102.84,81.67,271.0,-0.11,0.58,62.71,XR,NTT,CERAMICS
5,ANGELONE,3033.00,59.43,73.0,X-SC,7.09,223901.0,32895.0,24808.0,1.90,17.22,11.08,30.21,97.0,1.33,1.54,41.70,X40N,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.60
1,25,44.52
2,50,75.75


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    47.59
MC    27.77
LC    24.68
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.48
X40      20.52
X40N     12.24
XY25     10.06
XR        9.57
AR        9.00
OX40N     7.54
X200      1.80
SR        1.01
MH        0.82
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.25
X-MC    21.41
X-LC    19.80
M-SC    13.34
X-SC     8.55
H-MC     4.67
H-LC     2.86
L-SC     1.45
M-MC     1.36
M-LC     1.04
L-LC     0.98
L-MC     0.33
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.65,-4.33,38.07
IT,12.45,-16.63,79.06
FINANCE,11.92,-9.33,58.88
MISC,7.54,-19.02,79.06
PAINTS,6.03,-5.68,21.44
ELECTRICAL,5.56,-12.20,51.98
INSURANCE,4.52,3.49,30.53
AUTO,3.45,-14.91,125.09
PHARMA,3.43,-0.71,34.49


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3321329.0,20
XR,1271875.0,13
AR,1268756.0,10
X40,998886.0,14
OX40N,700045.0,10
X40N,694856.0,9
XY25,477013.0,7
SR,274369.0,2
MH,65530.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3492884.0,25
M-SC,1418088.0,16
X-LC,1198973.0,11
X-MC,1144021.0,15
X-SC,835421.0,8
H-MC,403229.0,3
L-SC,251545.0,3
H-LC,143336.0,2
M-LC,109741.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1169768.0      6
           AR         843812.0      5
           XR         801169.0      7
M-SC       XY24       740372.0      6
X-LC       XY24       503680.0      2
           X40        415585.0      5
X-MC       X40        413775.0      7
X-SC       XY24       376062.0      2
X-MC       XY24       347718.0      3
H-SC       OX40N      338236.0      4
X-SC       X40N       289833.0      4
H-SC       SR         274369.0      2
M-SC       OX40N      273737.0      5
H-MC       AR         219500.0      2
X-LC       X40N       204074.0      2
X-MC       X40N       200949.0      3
H-MC       XY24       183729.0      1
X-MC       XY25       181579.0      2
X-SC       X40        169526.0      2
L-SC       XR         163473.0      2
M-SC       XR         143374.0      2
           XY25       134743.0      1
           AR         125862.0      2
M-LC       XR         109741.0      1
L-SC       OX40N       88072.0      1
H-LC       AR          79582.0      1
X-LC       XY25        75634.0      2
H-SC       MH          65530.0      1
H-LC       X200        63754.0      1
L-MC       XR          54118.0      1
M-MC       XY25        48176.0      1
L-LC       XY25        36881.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 41.0 seconds
